# Ansys command output processing

You can extract the output of ``Ansys.send`` command in multiple ways. This example demonstrates these various methods.

In [1]:
from pansys import Ansys

In [2]:
a = Ansys(cleanup=True)

In [3]:
a.send('/prep7')

In [6]:
a.send("""
et,,185
et,,186
et,,187
""")

## The output property

After any send command is executed, if you access the ``output`` property of the ``Ansys`` object, you will get the output of the last command.

Example shown below.

In [41]:
a.send("etlist")
print(a.output)

etlist

 LIST ELEMENT TYPES FROM      1 TO      3 BY      1

 ELEMENT TYPE      1 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      2 IS SOLID186     3-D 20-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      3 IS SOLID187     3-D 10-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ  
  THREE-DIMENSIONAL MODEL

 PREP7:



## Live output

The output property is accessible only after the command is executed. So it becomes un-usable for commands which take a long time to execute and you want to see the progress live; eg: solution.

In such cases, you can use the ``silent`` keyword argument of the ``send`` command as shown below.

In [8]:
a.send('etlist', silent=False)

etlist

 LIST ELEMENT TYPES FROM      1 TO      3 BY      1

 ELEMENT TYPE      1 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      2 IS SOLID186     3-D 20-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      3 IS SOLID187     3-D 10-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

CURRENT NODAL DOF SET IS  UX    UY    UZ
THREE-DIMENSIONAL MODEL

PREP7:


## The ``output_function``

The send command also accepts another keyword arguement called the ``output_function`` which accepts a function. The output of the send command is passed to this function for further processing. By default, the ``output_function`` is set to the ``print`` funtion and hence, the output is printed out.

In the following example, a custom ``output_function`` is defined which will convert all the text into lower case.

In [9]:
def output_proc(l):
    print(l.lower())

In [12]:
a.send('etlist', silent=False, output_function=output_proc)

etlist

 list element types from      1 to      3 by      1

 element type      1 is solid185     3-d 8-node structural solid 
  keyopt( 1- 6)=        0      0      0        0      0      0
  keyopt( 7-12)=        0      0      0        0      0      0
  keyopt(13-18)=        0      0      0        0      0      0

 element type      2 is solid186     3-d 20-node structural solid
  keyopt( 1- 6)=        0      0      0        0      0      0
  keyopt( 7-12)=        0      0      0        0      0      0
  keyopt(13-18)=        0      0      0        0      0      0

 element type      3 is solid187     3-d 10-node structural solid
  keyopt( 1- 6)=        0      0      0        0      0      0
  keyopt( 7-12)=        0      0      0        0      0      0
  keyopt(13-18)=        0      0      0        0      0      0

current nodal dof set is  ux    uy    uz
three-dimensional model

prep7:


You can notice that all the output was converted to lowercase letters.

## Caveats of the ``output_function``

The data transferred to the output function is not line by line. It depends on the refresh rate of ANSYS output and how quickly python is reading it in.

The below example demonstrates this issue.

In [25]:
def output_proc2(l):
    print(l)
    print('---------------------')

In [38]:
a.send('etlist', silent=False, output_function=output_proc2)

etlist

 LIST ELEMENT TYPES FROM      1 TO      3 BY      1

 ELEMENT TYPE      1 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      2 IS SOLID186     3-D 20-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      3 IS SOLID187     3-D 10-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0
---------------------

---------------------
CURRENT NODAL DOF SET IS  UX    UY    UZ
---------------------
THREE-DIMENSIONAL MODEL
---------------------

-----------------

If the input to the ``output_function`` was line by line, you would have seen ``----------------------`` after every line. But that is not the case. This is because the output was read-in as chunks of multiple lines. So, if you want to process line by line, then you will have to split it yourselves by ``\r\n``. 

In [43]:
def output_proc3(l):
    for line in l.split('\r\n'):
        print(line)
        print('---------------------')

In [44]:
a.send('etlist', silent=False, output_function=output_proc3)

etlist
---------------------

---------------------
 LIST ELEMENT TYPES FROM      1 TO      3 BY      1
---------------------

---------------------
 ELEMENT TYPE      1 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
---------------------
  KEYOPT( 1- 6)=        0      0      0        0      0      0
---------------------
  KEYOPT( 7-12)=        0      0      0        0      0      0
---------------------
  KEYOPT(13-18)=        0      0      0        0      0      0
---------------------

---------------------
 ELEMENT TYPE      2 IS SOLID186     3-D 20-NODE STRUCTURAL SOLID
---------------------
  KEYOPT( 1- 6)=        0      0      0        0      0      0
---------------------
  KEYOPT( 7-12)=        0      0      0        0      0      0
---------------------
  KEYOPT(13-18)=        0      0      0        0      0      0
---------------------

---------------------
 ELEMENT TYPE      3 IS SOLID187     3-D 10-NODE STRUCTURAL SOLID
---------------------
  KEYOPT( 1- 6)=        0      

If you dont want live updates, you can always use the ``output`` variable.

In [36]:
for line in a.output.split('\r\n'):
    if 'element type ' in line.lower():
        temp = [x.strip() for x in line.split()]
        print(temp[2], temp[4], temp[5], temp[6])

1 SOLID185 3-D 8-NODE
2 SOLID186 3-D 20-NODE
3 SOLID187 3-D 10-NODE


## Large outputs

If the output of your command is large and you dont want to keep the entire thing in memory, then there is another method called ``get_output`` which will write the output of any command passed to it to a file.

In [45]:
etypes = a.get_output('etlist')

In [46]:
etypes

'/home/yy53393/github/pansys/docs/_src/examples/pansys_20180712125145/out.out'

In [48]:
!cat $etypes


 LIST ELEMENT TYPES FROM      1 TO      3 BY      1

 ELEMENT TYPE      1 IS SOLID185     3-D 8-NODE STRUCTURAL SOLID 
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      2 IS SOLID186     3-D 20-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 ELEMENT TYPE      3 IS SOLID187     3-D 10-NODE STRUCTURAL SOLID
  KEYOPT( 1- 6)=        0      0      0        0      0      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    UZ  
  THREE-DIMENSIONAL MODEL


As you can see, the ``get_output`` command wrote the output to a file called ``out.out`` in your current working directory. This file will get overwritten every time you use ``get_output``. If you want to keep the output file, you should set the ``persist`` keyword argument to true which will produce a unique filename every time.

In [49]:
etypes = a.get_output('etlist', persist=True)

In [50]:
etypes

'/home/yy53393/github/pansys/docs/_src/examples/pansys_20180712125145/e562762c-155f-466c-8970-3c0ee709c2ea'